In [63]:
import pandas as pd
import re

In [122]:
df = pd.read_csv("biorxiv.csv")

In [123]:
#Affiliatin_nums: Replace NAs with 1, for single affiliation.
#Affiliation_nums: Split into list
df.affiliation_nums = df.affiliation_nums.fillna('1')
df.affiliation_nums = df.affiliation_nums.apply(str.split,sep=',')

In [124]:
def clean_affiliation(x):
    x = x.split(';')
    for i in range(len(x)):
        x[i] = re.sub('(\\n)(\\t)+','',x[i])
    return(x)
df.affiliation_text = df.affiliation_text.fillna('')
df.affiliation_text = df.affiliation_text.apply(clean_affiliation)

In [120]:
df

,affiliation_nums,affiliation_text,author_names,date_posted,doi,downloads,paper_title,subject_area
0,[1],"[, University of Southern California]","James H Joly,Brandon TL Chew,Nicholas Alexande...","\n Posted October 07, 2020.",https://doi.org/10.1101/2020.10.05.327429,"<tr class=""odd""><td>Oct 2020</td><td>500</td><...",The landscape of metabolic pathway dependencie...,"Cancer Biology,\n"
1,"[1, 3, 4, 1, 2, 3, 2, 3, 2, 3, 2, 2, 3, 2, 3, ...","[, Glasgow G61 1QH, United Kingdom,, Glasgow G...","Mate Naszai,Yachuan Yu,Alvaro R Fernandez,Emma...","\n Posted October 08, 2020.",https://doi.org/10.1101/2020.10.07.329607,"<tr class=""odd""><td>Oct 2020</td><td>343</td><...",RAL GTPases mediate EGFR/MAPK signalling-drive...,"Cancer Biology,\n"
2,"[1, 4, 2, 3, 3, 2, 2]","[, H. Lee Moffitt Cancer Center & Research Ins...","Jeffrey West,Maximilian Strobl,Cole Armagost,R...","\n Posted October 09, 2020.",https://doi.org/10.1101/2020.10.08.331678,"<tr class=""odd""><td>Oct 2020</td><td>280</td><...",Antifragile therapy,"Cancer Biology,\n"
3,"[1, 10, 1, 1, 1, 1, 2, 1, 1, 1, 3, 3, 4, 4, 4,...","[, University of Cambridge, ,, Cancer Evolutio...","Filipe Correia Martins,Dominique-Laurent Coutu...","\n Posted October 12, 2020.",https://doi.org/10.1101/2020.10.04.325365,"<tr class=""odd""><td>Oct 2020</td><td>768</td><...",Somatic chromosomal number alterations affecti...,"Cancer Biology,\n"
4,"[1, 2, 3, 1, 4, 5, 4, 6, 7, 1, 9, 8]","[, Helen Diller Comprehensive Cancer Center, U...","Andrew L Wolfe,Qingwen Zhou,Eneda Toska,Jacque...","\n Posted October 13, 2020.",https://doi.org/10.1101/2020.10.13.337998,"<tr class=""odd""><td>Oct 2020</td><td>353</td><...","UDP-glucose pyrophosphorylase 2, a regulator o...","Cancer Biology,\n"
...,...,...,...,...,...,...,...,...
3634,"[1, +, #, 1, 2, 3, 4, 2, 5, 1, *]","[, Tucson, Arizona,, Tucson, Arizona,, Tucson,...","Brenna A. Rheinheimer,Ronald L. Heimark,Adam D...","\n Posted September 30, 2020.",https://doi.org/10.1101/2020.09.29.318873,"<tr class=""odd""><td>Sep 2020</td><td>194</td><...","Cell intrinsic signaling in , mutant pancreati...","Cancer Biology,\n"
3635,"[1, 2, 1, 1, 1, 3, 4, 1, 1, 1, 1, 1, *, 5, 6, ...",[],"Lei Huang,Xiao-Ou Zhang,Odette Verdejo-Torres,...","\n Posted September 30, 2020.",https://doi.org/10.1101/2020.08.12.246660,"<tr class=""odd""><td>Aug 2020</td><td>703</td><...",Protein arginine methyltransferase 5 promotes ...,"Cancer Biology,\n"
3636,"[1, 5, 2, 5, 2, 5, 2, 1, 5, 3, 1, 4, 1, 5, *]","[, Houston TX,, Houston TX,, Houston TX,, Hous...","Emilly S Villodre,Yun Gong,Lei Huo,Esther C Yo...","\n Posted September 28, 2020.",https://doi.org/10.1101/2020.09.25.313817,"<tr class=""odd""><td>Sep 2020</td><td>372</td><...",NDRG1 expression is an independent prognostic ...,"Cancer Biology,\n"
3637,"[1, #, 1, #, 2, 1, 3, 3, 3, 3, 3, *, 1, *]","[, Beijing 100037, China,, Chengdu 610041, Chi...","Xuanyu Liu,Meng Yuan,Qinqin Xiang,Wen Chen,Zhu...","\n Posted September 28, 2020.",https://doi.org/10.1101/2020.09.27.315911,"<tr class=""odd""><td>Sep 2020</td><td>248</td><...",Single-cell RNA-seq of the stromal vascular fr...,"Cancer Biology,\n"


In [17]:
type(affs.apply(str.split,sep=',')[1])

list

In [22]:
affs = affs.apply(str.split,sep=',')

In [23]:
affs

0                                                   [1]
1     [1, 3, 4, 1, 2, 3, 2, 3, 2, 3, 2, 2, 3, 2, 3, ...
2                                 [1, 4, 2, 3, 3, 2, 2]
3     [1, 10, 1, 1, 1, 1, 2, 1, 1, 1, 3, 3, 4, 4, 4,...
4                  [1, 2, 3, 1, 4, 5, 4, 6, 7, 1, 9, 8]
5                                                   [1]
6     [1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 4, ...
7                                 [1, 1, 1, 2, 1, 1, 3]
8                  [1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3]
9     [1, 2, 2, 2, 3, 3, 4, 5, 6, 7, 8, 6, 9, 6, 5, ...
10    [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 3, 1, 2, 1, ...
11    [1, 1, 1, 1, 1, 2, 2, 1, 3, 1, 1, 3, 3, 4, 5, ...
12                                            [1, 3, 2]
13                       [1, *, 1, 2, 3, 4, 5, 6, 7, 4]
14                          [1, 1, 2, 3, 4, 2, 1, 5, *]
15    [1, 2, *, 1, 3, *, 4, 5, 1, 3, 1, 1, 6, #, 7, ...
16                                [1, 2, 3, 4, 1, 1, *]
17                                            [1

In [26]:
affs_type=df.affiliation_nums.apply(type)

In [36]:
affs_type[affs_type==float]

22      <class 'float'>
171     <class 'float'>
180     <class 'float'>
197     <class 'float'>
202     <class 'float'>
             ...       
1394    <class 'float'>
1408    <class 'float'>
1430    <class 'float'>
1438    <class 'float'>
1817    <class 'float'>
Name: affiliation_nums, Length: 75, dtype: object

In [94]:
test = df.affiliation_text[2]
test

'\n\t\t\t\t, H. Lee Moffitt Cancer Center & Research Institute, Tampa, Florida;\n\t\t\t,\n\t\t\t\t, H. Lee Moffitt Cancer Center & Research Institute;\n\t\t\t,\n\t\t\t\t, Air Force Academy\n\t\t\t'

In [95]:
test = test.split(';')

In [103]:
re.sub('(\\n)(\\t)+','',test[0])

', H. Lee Moffitt Cancer Center & Research Institute, Tampa, Florida'

In [104]:
def clean_affiliation(x):
    x = x.split(';')
    for i in range(len(x)):
        x[i] = re.sub('(\\n)(\\t)+','',x[i])
    return(x)

In [108]:
test = df.affiliation_text.head()
test = test.apply(clean_affiliation)
test[0]

In [93]:
re.match(',',test[2].strip()).lastindex

In [113]:
test[3]

[', University of Cambridge',
 ',, Cancer Evolution and Genome Instability Laboratory, Francis Crick Institute, London, UK',
 ',, Cambridge University Hospitals',
 ',, Astrazeneca',
 ',, The Francis Crick Institute',
 ',, University College of London',
 ',, Francis Crick Institute',
 ',, BC Cancer Agency',
 ',, Imperial College of London']

In [116]:
df.affiliation_text

0       \n\t\t\t\t, University of Southern California\...
1       , Glasgow G61 1QH, United Kingdom,, Glasgow G6...
2       \n\t\t\t\t, H. Lee Moffitt Cancer Center & Res...
3       \n\t\t\t\t, University of Cambridge;\n\t\t\t,\...
4       \n\t\t\t\t, Helen Diller Comprehensive Cancer ...
                              ...                        
3634    , Tucson, Arizona,, Tucson, Arizona,, Tucson, ...
3635                                                  NaN
3636    , Houston TX,, Houston TX,, Houston TX,, Houst...
3637    , Beijing 100037, China,, Chengdu 610041, Chin...
3638    , Auckland, New Zealand, Auckland, New Zealand...
Name: affiliation_text, Length: 3639, dtype: object